# Nettoyage de "***la Grosse Bertha***"

### Import modules

In [1]:
import sqlite3

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

### Connexion à "***la Grosse Bertha_1.1***" en **LOCAL** sur mon poste

In [2]:
path = 'D:/tmp storage/Project#3/SQLite3 db/GrosseBertha_1.1.db'
db_conn = sqlite3.connect(path)

## Exploration des tables

### Filtrage des films
On cherche à ne garder dans la database **que les films présents initialement dans la base de données de notre client**.
On supprimera donc toutes les entrées de la base IMDB qui ne sont pas dans la base du client.

#### Table **links**


In [ ]:
query = '''
SELECT * FROM links
LIMIT 1
'''
pd.read_sql(query, db_conn)

La table **links** comporte les **id** des films utilisés dans la base client et dans la base IMDB.
___
Première ligne de la table **movies** correnspondant au film "*Toy Story*"


In [ ]:
query = '''
SELECT * FROM movies
LIMIT 1
'''
pd.read_sql(query, db_conn)

La ligne de la table **imdb_title_basics** correspondante au film "*Toy Story*" (l'équivalent de la table movies pour la base imdb)

In [ ]:
query = '''
SELECT * FROM imdb_title_basics
WHERE primaryTitle == 'Toy Story' AND titleType == 'movie'
'''
pd.read_sql(query, db_conn)

On constate que **`tconst`**, qui représente l'id du film dans cette table, n'a pas tout à fait le format attendu..

|**id dans links** | **id dans imdb** |
|------------------|------------------|
| 114709           | tt0114709         |

Il faut donc convertir les **id** de la table **links** pour qu'ils correspondent à ceux utilisés dans la base IMDB
___
On créer un DF à partir de **links**

In [9]:
query = '''
SELECT * FROM links
'''
links = pd.DataFrame(pd.read_sql(query, db_conn))

In [ ]:
links.info()

In [10]:
links.shape

(58098, 3)

On ajoute des **`00`** pour avoir 7 chiffres et **`tt`** à chaque **`imdbId`**

In [ ]:
# links['imdbId'] = links['imdbId'].apply(lambda x: str(x).zfill(7))
# links['imdbId'] = links['imdbId'].apply(lambda x: 'tt' + x)

In [ ]:
links.head()

In [ ]:
links.tail()

Et on remplace la table **links** par le DataFrame ***links***

In [ ]:
# links.to_sql('links', db_conn, if_exists = 'replace', index = False)

In [ ]:
query = '''
SELECT * FROM links
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape

On constate que la table link comporte maintenant le double de lignes. Il faut donc supprimer


## Filtrage des films
On souhaite ne garder que les films qui sont dans la base de données de notre client pour ne pas proposer des films qu'il ne pocède pas.

### SELECT
On sélectionne tous les **films de la base IMDB** dont l'identifiant se trouve également dans la table **links**

In [11]:
query = '''
SELECT * FROM imdb_title_basics
WHERE tconst IN
    (SELECT imdbId
     FROM links)
'''
imdb_mouvies = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_mouvies.shape

(58016, 9)

Il y a donc 58016 films en commun (il semblerait que certains films n'ait pas leur id dans la table IMBD)
___
### INSERT dans la BDD
On remplace la table **`imdb_title_basics`** par le DF qui ne contient que les films client.

In [13]:
imdb_mouvies.to_sql('imdb_title_basics', db_conn, if_exists = 'replace', index = False)

In [15]:
query = '''
SELECT * FROM imdb_title_basics
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape

(58016, 9)

## Nettoyage des "ratings"

### SELECT
On ne sélectionne que les ratings portant sur les films qui sont dans notre BDD

In [17]:
query = '''
SELECT * FROM imdb_title_ratings Ir
WHERE Ir.tconst IN (
    SELECT imdbId FROM links
    )
'''
imdb_ratings = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_ratings.shape

(57928, 3)

### Import to BDD

In [18]:
imdb_ratings.to_sql('imdb_title_ratings', db_conn, if_exists = 'replace', index = False)

In [19]:
query = '''
SELECT * FROM imdb_title_ratings
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape


(1036256, 3)